In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

思路：可以增加的features
1. 是否同一ip地址多个帐号
2. 登录的平均间隔时间、最短间隔时间、最大间隔时间
3. 一天中的交易次数

In [3]:
trade_df = pd.read_csv('./data/Risk_Detection_Qualification/t_trade.csv', index_col='rowkey')
login_df = pd.read_csv('./data/Risk_Detection_Qualification/t_login.csv', index_col='log_id')
trade_test_df = pd.read_csv('./data/Risk_Detection_Qualification/t_trade_test.csv', index_col='rowkey')
login_test_df = pd.read_csv('./data/Risk_Detection_Qualification/t_login_test.csv', index_col='log_id')

In [4]:
# 得到一列不同的值
# login_df['result'].unique()
# login_df['id'].unique()
# len(login_df['city'].unique()), len(login_df.index)
# login_df['is_risk'].count()
trade_df.shape, trade_df[trade_df['is_risk']==0].shape, trade_df[trade_df['is_risk']==1].shape

((132719, 3), (129076, 3), (3643, 3))

In [5]:
trade_df_by_id = trade_df['is_risk'].groupby(trade_df['id'])

id_risk_trade   = trade_df[trade_df['is_risk']==1]['id']
id_nonrisk_trade = trade_df[trade_df['is_risk']==0]['id']
checked_id = set(id_risk) & set(id_nonrisk)
len(id_risk), len(id_nonrisk), len(checked_id)

(3643, 129076, 455)

In [14]:
login_df_by_id = login_df['ip'].groupby(login_df['id']).count()
#print(login_df_by_id)
# '175732' in login_df['id']
login_df['id']

log_id
2.618369e+15    116825
9.969739e+14    109987
8.123343e+15    116825
3.265677e+14    116825
6.684659e+15    116825
8.828481e+15    116825
6.071119e+15    116825
8.525176e+15    109987
3.120145e+15    116825
5.367905e+15    109987
3.378803e+15    109987
4.255256e+16    109987
9.536343e+15    109987
3.776181e+15    109987
8.824729e+15    109987
3.315790e+16    153075
5.175846e+15    153075
6.456778e+15    153075
1.912764e+15    153075
2.349293e+16    153075
3.722752e+15    153075
3.030546e+15     20004
1.308425e+16     29583
7.065851e+15     29583
8.592228e+15     29583
3.122594e+16    165362
4.427012e+16     91358
6.737734e+14    153075
5.275490e+15     18169
7.227700e+15     22816
                 ...  
1.102321e+16     59434
6.801987e+14      3608
3.643520e+16      3608
5.601965e+15      4855
9.184832e+15     47845
4.288803e+15     47845
8.548634e+15     47845
6.652598e+15     47845
7.117072e+15     47845
5.932818e+15     65043
9.354748e+15     65043
9.620718e+15     65043
1.79

In [11]:
'175766' in id_risk
'175766' in id_nonrisk

False